In [1]:
import pandas as pd
import os

In [2]:
def read_feature_file_as_list(filename: str):
    feature_file = open(filename, 'r', encoding='utf-8')
    features = [line.strip() for line in feature_file.readlines()]
    return features


def extract_data(set_type, language):
    base_path = f'./../data/{language}'
    base_features_path = f'{base_path}/{set_type}/{set_type}'

    questions = read_feature_file_as_list(f'{base_features_path}.question')
    answers = read_feature_file_as_list(f'{base_features_path}.answer')
    code = read_feature_file_as_list(f'{base_features_path}.code')
    original_code = read_feature_file_as_list(f'{base_features_path}.code.original')

    df = pd.DataFrame(
        {
            'questions': questions,
            'answers': answers,
            'code': code,
            'original_code': original_code
        }
    )

    path = f'{base_path}/datasets'
    dir_exists = os.path.exists(path)
    if not dir_exists:
        os.makedirs(path)
    df.to_csv(f'{path}/{set_type}.csv')

In [3]:
# Java
extract_data(set_type="train", language="java")
extract_data(set_type="dev", language="java")
extract_data(set_type="test", language="java")

# Python
extract_data(set_type="train", language="python")
extract_data(set_type="dev", language="python")
extract_data(set_type="test", language="python")